In [257]:
from urllib import request
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
pd.set_option('display.max_rows', None)
import time


In [90]:
html = requests.get('https://urtrip.jp/tds-past-info/?rm=20230602#cal')
soup = bs(html.text,"html.parser")

<!DOCTYPE html>
<!--[if lt IE 7]>
<html class="ie6" lang="ja"
	prefix="og: https://ogp.me/ns#" > <![endif]-->
<!--[if IE 7]>
<html class="i7" lang="ja"
	prefix="og: https://ogp.me/ns#" > <![endif]-->
<!--[if IE 8]>
<html class="ie" lang="ja"
	prefix="og: https://ogp.me/ns#" > <![endif]-->
<!--[if gt IE 8]><!-->
<html lang="ja" prefix="og: https://ogp.me/ns#">
 <!--<![endif]-->
 <head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# article: http://ogp.me/ns/article#">
  <meta charset="utf-8"/>
  <meta content="width=device-width,initial-scale=1.0,user-scalable=yes" name="viewport"/>
  <meta content="telephone=no" name="format-detection"/>
  <link href="https://urtrip.jp/feed/" rel="alternate" title="ヨーロッパ・アジア旅行に便利な情報サイト RSS Feed" type="application/rss+xml"/>
  <link href="https://urtrip.jp/xmlrpc.php" rel="pingback"/>
  <!--[if lt IE 9]>
		<script src="http://css3-mediaqueries-js.googlecode.com/svn/trunk/css3-mediaqueries.js"></script>
		<script src="https://urtrip.jp/wp-content/

In [184]:
wating_time_record = pd.DataFrame()

In [233]:
date_str = '20220529' 

html = requests.get('https://urtrip.jp/tds-past-info/?rm='+str(date_str)+'#cal')
soup = bs(html.text,"html.parser")

table_list = soup.find_all("tbody", attrs={"class":"t_cool"})[:5]

waiting_time = pd.DataFrame(columns=['時間'])

for table in table_list:
    column_html = table.find_all(attrs = {"class":"t_cool_h"})[0]
    col = []
    for i in column_html.find_all("th"):
        col.append(i.get_text().replace('\n', '').replace('\r', ''))

    row_html = table.find_all("tr", attrs = {"class":"t_cool_b date_width"})

    table_list = []
    for r in row_html:
        row = []
        ele_html = r.find_all(['th','td'])
        for e in ele_html:
            row.append(e.get_text())
        table_list.append(row)
    tmp_table = pd.DataFrame(table_list)
    tmp_table.columns = col
    waiting_time = pd.merge(waiting_time, tmp_table, how='outer', on='時間')
waiting_time.to_csv('./disneysea_waitingtime_'+str(date_str)+'.csv', encoding='utf-8')

#stock用のデータを作る
waiting_time.index = waiting_time['時間']
waiting_time_db = waiting_time.drop('時間', axis =1).stack().reset_index()
waiting_time_db.columns = ['time','name_atrc', 'w_time']
waiting_time_db['date'] = date_str
waiting_time_db['w_time_num'] = pd.to_numeric(waiting_time_db['w_time'],errors='coerce')
wating_time_record = pd.concat([wating_time_record,waiting_time_db])

In [234]:
wating_time_record = wating_time_record.drop_duplicates()
wating_time_record

,time,name_atrc,w_time,date,w_time_num
0,09:00,ソアリン,－,20230506,NaN
1,09:00,トイ・ストーリ・マニア,－,20230506,NaN
2,09:00,タワー・オブ・テラー,－,20230506,NaN
3,09:00,センター・オブ・ジアース,－,20230506,NaN
4,09:00,インディジョーンズ,－,20230506,NaN
...,...,...,...,...,...
1243,20:45,スチーマーライン(ポート),案内終了,20220529,NaN
1244,20:45,スチーマーライン(ハーバー),案内終了,20220529,NaN
1245,20:45,スチーマーライン(ロスト),案内終了,20220529,NaN
1246,20:45,スチーマーライン(フロント),案内終了,20220529,NaN


In [223]:
g = wating_time_record.query('20220601 <= date.astype("int") <= 20220630')[['name_atrc','w_time_num']].dropna().groupby('name_atrc')
g.describe()

w_time_num                                                  \
                      count        mean        std  min   25%    50%    75%   
name_atrc                                                                     
アクアトピア                337.0   16.201780   8.414181  5.0  10.0   15.0   20.0   
インディジョーンズ             356.0   31.643258  13.291286  5.0  20.0   35.0   40.0   
キャラバンカルーセル            350.0    5.485714   2.015292  5.0   5.0    5.0    5.0   
シンドバッド・ストーリーブック       380.0    5.526316   1.660290  5.0   5.0    5.0    5.0   
ジャスミンのフライングカーペット      361.0    9.473684   4.472136  5.0   5.0   10.0   10.0   
ジャンピン・ジェリーフィッシュ       384.0    9.427083   4.986655  5.0   5.0   10.0   15.0   
スカットルのスクーター           383.0   10.678851   4.860219  5.0   5.0   10.0   15.0   
スチーマーライン(ハーバー)        183.0    5.792350   2.236202  5.0   5.0    5.0    5.0   
スチーマーライン(フロント)        180.0    6.055556   2.641995  5.0   5.0    5.0    5.0   
スチーマーライン(ポート)         271.0    9.944649   4.644459  5.0   5.0   10.0   15.0   
スチーマーライン(ロスト)         183.0    8.087432   4.792738  5.0   5.0    5.0   10.0   
センター・オブ・ジアース          367.0   44.305177  14.049855  5.0  35.0   45.0   55.0   
ソアリン                  348.0  110.876437  29.418362  5.0  90.0  110.0  130.0   
タワー・オブ・テラー            353.0   72.266289  16.370790  5.0  60.0   70.0   80.0   
タートル・トーク              343.0   22.886297   7.333446  5.0  20.0   20.0   30.0   
トイ・ストーリ・マニア           359.0   74.470752  21.710992  5.0  60.0   75.0   90.0   
ニモ＆フレンズ・シーライダ         382.0   18.664921   5.811815  5.0  15.0   15.0   20.0   
フランダーのフライングフィッシュ      381.0   14.632546   6.365730  5.0  10.0   15.0   20.0   
ブローフィッシュバルーン          383.0   16.501305   6.828442  5.0  10.0   15.0   20.0   
マジックランプシアター           310.0   25.000000   7.296317  5.0  20.0   25.0   30.0   
レイジングスピリッツ            363.0   30.110193  11.241398  5.0  25.0   30.0   40.0   
レールウェイ(フロント)          272.0    9.797794   5.195044  5.0   5.0   10.0   15.0   
ワールプール                384.0   12.096354   5.744207  5.0   5.0   15.0   15.0   
ヴェネツィアン・ゴンドラ          316.0   28.196203  13.095720  0.0  20.0   30.0   35.0   
海底２万マイル               380.0    9.421053   5.678881  5.0   5.0    5.0   10.0   

                         
                    max  
name_atrc                
アクアトピア             35.0  
インディジョーンズ          60.0  
キャラバンカルーセル         15.0  
シンドバッド・ストーリーブック    15.0  
ジャスミンのフライングカーペット   20.0  
ジャンピン・ジェリーフィッシュ    25.0  
スカットルのスクーター        20.0  
スチーマーライン(ハーバー)     20.0  
スチーマーライン(フロント)     20.0  
スチーマーライン(ポート)      20.0  
スチーマーライン(ロスト)      25.0  
センター・オブ・ジアース       70.0  
ソアリン              180.0  
タワー・オブ・テラー        110.0  
タートル・トーク           45.0  
トイ・ストーリ・マニア       130.0  
ニモ＆フレンズ・シーライダ      45.0  
フランダーのフライングフィッシュ   25.0  
ブローフィッシュバルーン       30.0  
マジックランプシアター        50.0  
レイジングスピリッツ         60.0  
レールウェイ(フロント)       25.0  
ワールプール             25.0  
ヴェネツィアン・ゴンドラ       75.0  
海底２万マイル            30.0

In [224]:
g = wating_time_record.query('20230501 <= date.astype("int") <= 20230531')[['name_atrc','w_time_num']].dropna().groupby('name_atrc'
g.describe()

w_time_num                                                    \
                      count        mean        std   min    25%    50%    75%   
name_atrc                                                                       
アクアトピア                302.0   21.225166  12.144847   5.0    5.0   20.0   30.0   
インディジョーンズ             358.0   38.435754  19.462240   5.0   25.0   40.0   55.0   
キャラバンカルーセル            337.0    7.032641   3.632132   5.0    5.0    5.0   10.0   
シンドバッド・ストーリーブック       365.0    6.479452   3.356598   5.0    5.0    5.0    5.0   
ジャスミンのフライングカーペット      351.0   12.606838   7.780882   5.0    5.0   10.0   20.0   
ジャンピン・ジェリーフィッシュ       352.0   12.613636   6.325784   5.0    5.0   15.0   20.0   
スカットルのスクーター           352.0   14.005682   7.586536   5.0    5.0   15.0   20.0   
スチーマーライン(ハーバー)        194.0    7.164948   3.903447   5.0    5.0    5.0   10.0   
スチーマーライン(フロント)        164.0   11.006098   7.053257   5.0    5.0    5.0   20.0   
スチーマーライン(ポート)         319.0   11.677116   5.726399   5.0    5.0   10.0   15.0   
スチーマーライン(ロスト)         242.0   10.743802   8.240703   5.0    5.0    5.0   20.0   
センター・オブ・ジアース          333.0   60.390390  25.352886   5.0   45.0   65.0   80.0   
ソアリン                  334.0  118.667665  41.497209  25.0  100.0  130.0  150.0   
タワー・オブ・テラー            357.0   38.753501  20.235295  10.0   20.0   40.0   50.0   
タートル・トーク              369.0   29.010840  10.368739  15.0   20.0   30.0   35.0   
トイ・ストーリ・マニア           335.0   96.522388  32.547689  20.0   72.5  100.0  120.0   
ニモ＆フレンズ・シーライダ         360.0   32.569444  12.852090  15.0   20.0   35.0   45.0   
ビッグシティ・ヴィークル          145.0    6.172414   2.826818   5.0    5.0    5.0    5.0   
フランダーのフライングフィッシュ      367.0   20.899183  10.964831   5.0   10.0   20.0   30.0   
ブローフィッシュバルーン          369.0   20.826558  10.875144   5.0   10.0   20.0   30.0   
マジックランプシアター           329.0   23.480243   7.395640  10.0   20.0   20.0   30.0   
レイジングスピリッツ             92.0   23.858696  19.696924   5.0    5.0   12.5   45.0   
レールウェイ(フロント)          319.0   12.742947   6.593527   5.0    5.0   15.0   20.0   
ワールプール                353.0   16.983003   8.686768   5.0   10.0   15.0   25.0   
ヴェネツィアン・ゴンドラ          193.0   18.860104  11.913145   5.0    5.0   20.0   30.0   
海底２万マイル               365.0   14.369863   9.162014   5.0    5.0   15.0   20.0   

                         
                    max  
name_atrc                
アクアトピア             45.0  
インディジョーンズ          75.0  
キャラバンカルーセル         25.0  
シンドバッド・ストーリーブック    20.0  
ジャスミンのフライングカーペット   30.0  
ジャンピン・ジェリーフィッシュ    30.0  
スカットルのスクーター        30.0  
スチーマーライン(ハーバー)     20.0  
スチーマーライン(フロント)     20.0  
スチーマーライン(ポート)      25.0  
スチーマーライン(ロスト)      40.0  
センター・オブ・ジアース      105.0  
ソアリン              220.0  
タワー・オブ・テラー        110.0  
タートル・トーク           55.0  
トイ・ストーリ・マニア       165.0  
ニモ＆フレンズ・シーライダ      60.0  
ビッグシティ・ヴィークル       15.0  
フランダーのフライングフィッシュ   50.0  
ブローフィッシュバルーン       40.0  
マジックランプシアター        50.0  
レイジングスピリッツ         55.0  
レールウェイ(フロント)       30.0  
ワールプール             35.0  
ヴェネツィアン・ゴンドラ       40.0  
海底２万マイル            40.0

In [245]:
g = wating_time_record.query('20230501 <= date.astype("int") <= 20230531')[['name_atrc','time','w_time_num']].dropna().groupby(['name_atrc','time'])
g_mean = g.mean()

In [253]:
g_mean.pivot_table(index = 'time', columns = 'name_atrc').to_csv('./DisneySea_wtime_ave_202305.csv',encoding='shift-jis')

In [260]:
for date in pd.date_range(start='1/1/2018', end='31/12/2018'):
    print(date.strftime('%Y%m%d'))
#     time.sleep(10)

20180101
20180102
20180103
20180104
20180105
20180106
20180107
20180108
20180109
20180110
20180111
20180112
20180113
20180114
20180115
20180116
20180117
20180118
20180119
20180120
20180121
20180122
20180123
20180124
20180125
20180126
20180127
20180128
20180129
20180130
20180131
20180201
20180202
20180203
20180204
20180205
20180206
20180207
20180208
20180209
20180210
20180211
20180212
20180213
20180214
20180215
20180216
20180217
20180218
20180219
20180220
20180221
20180222
20180223
20180224
20180225
20180226
20180227
20180228
20180301
20180302
20180303
20180304
20180305
20180306
20180307
20180308
20180309
20180310
20180311
20180312
20180313
20180314
20180315
20180316
20180317
20180318
20180319
20180320
20180321
20180322
20180323
20180324
20180325
20180326
20180327
20180328
20180329
20180330
20180331
20180401
20180402
20180403
20180404
20180405
20180406
20180407
20180408
20180409
20180410
20180411
20180412
20180413
20180414
20180415
20180416
20180417
20180418
20180419
20180420
20180421
2

In [ ]:
date_str = '20220529' 

html = requests.get('https://urtrip.jp/tds-past-info/?rm='+str(date_str)+'#cal')
soup = bs(html.text,"html.parser")

table_list = soup.find_all("tbody", attrs={"class":"t_cool"})[:5]

waiting_time = pd.DataFrame(columns=['時間'])

for table in table_list:
    column_html = table.find_all(attrs = {"class":"t_cool_h"})[0]
    col = []
    for i in column_html.find_all("th"):
        col.append(i.get_text().replace('\n', '').replace('\r', ''))

    row_html = table.find_all("tr", attrs = {"class":"t_cool_b date_width"})

    table_list = []
    for r in row_html:
        row = []
        ele_html = r.find_all(['th','td'])
        for e in ele_html:
            row.append(e.get_text())
        table_list.append(row)
    tmp_table = pd.DataFrame(table_list)
    tmp_table.columns = col
    waiting_time = pd.merge(waiting_time, tmp_table, how='outer', on='時間')
waiting_time.to_csv('./disneysea_waitingtime_'+str(date_str)+'.csv', encoding='utf-8')

#stock用のデータを作る
waiting_time.index = waiting_time['時間']
waiting_time_db = waiting_time.drop('時間', axis =1).stack().reset_index()
waiting_time_db.columns = ['time','name_atrc', 'w_time']
waiting_time_db['date'] = date_str
waiting_time_db['w_time_num'] = pd.to_numeric(waiting_time_db['w_time'],errors='coerce')
wating_time_record = pd.concat([wating_time_record,waiting_time_db])